<a href="https://colab.research.google.com/github/kutyadog/ai_notebooks/blob/main/Ai_Rag_Basics_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Answering using Embeddings - AUG 2024

An extremely simple AI chatbot (using RAG) using completely unique content (that I just now created to ensure that the base model does not have the information already).

In [ ]:
!pip install openai numpy pandas tiktoken pinecone-client datasets

In [ ]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
import os

COMPLETIONS_MODEL = "gpt-4o-mini"
EMBEDDING_MODEL = "text-embedding-ada-002"

from google.colab import userdata


openai.organization = userdata.get('OPENAI_ORG')
openai.api_key = userdata.get('OPENAI_API_KEY')


In [ ]:
prompt = "Who is DargumagaX?"

# Use chat.completions.create for chat models
response = openai.chat.completions.create(
    model=COMPLETIONS_MODEL,
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0,
    max_tokens=300
)
# Extract the content from the response
answer = response.choices[0].message.content
print(answer)

As of my last knowledge update in October 2023, there is no widely recognized figure or entity known as "DargumagaX." It's possible that it could refer to a username, a character in a game, or a content creator on platforms like Twitch or YouTube. If you have more context or specific details about DargumagaX, I might be able to provide more relevant information.


GPT does not know who DargumagaX is, because I just made him up.

Luckily, newer models seem to hallucinate less often than the previous models had. The AI responsding with "I don't know" is preferred over a convincing hallucination.

Now Lets see if we can guide our AI to answer with a more standard answer.


In [ ]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: Who is DargumagaX?
A:"""

response = openai.chat.completions.create(
    model=COMPLETIONS_MODEL,
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0,
    max_tokens=300
)
# Extract the content from the response
answer = response.choices[0].message.content
print(answer)

Sorry, I don't know.


Thats even better!

Now lets provide extra contextual information in the prompt. When the total required context is short, we can include it in the prompt directly.

In [ ]:
prompt = """Answer the question as truthfully as possible using the provided text,
and if the answer is not contained within the text below, say "I don't know"
Keep answers simple and short.

Context:
DargumagaX is a super hero alien.
His home planet is DilsdIlwkdiwK9.
His real name is ighaphamadulaboggaDing, but he prefers to go by DargumagaX.
He has the power to fly, shoot lasers from his eyes, and is super strong.
He was born with his powers, they are part of his natural abilities.
He is a kind and gentle creature, but he is also very brave, flatulent and determined.
DargumagaX's weakness is Bughaphaknite, a rare mineral found only in his home solar system.
His primary enemy are the ParPukas, an evil alien species from the planet Biffron, who want to conquer the universe and 'eat all the monkeys'.
DargumagaX was born on the planet DilsdIlwkdiwK9, which was destroyed by the evil aliens from Biffron.
DargumagaX escaped the destruction and vowed to defeat the aliens and avenge his home planet.
His arch-nemesis is Lord Blart, who is an evil ParPukas dictator.
His favorite food is a special type of bean that grew on his home planet called Drudigan Beans.
He says the closest thing to them on earth is the Geoduck Clam (google it).
His biggest fear is that he will fail to protect the universe from the ParPukas.
He is terribly afraid of kittens.
DargumagaX is a member of the Intergalactic League for Somewhat Fair Justice, a group of superheroes from different planets who work together to somewhat protect the universe.
Unlike some other boring superheros, he has no secret identity.
He is known to everyone as DargumagaX.
His favorite color is Drab dark brown, otherwise known as 'Pantone 448 C'.
He likes exploring different planets, learning about new cultures, and collecting Navel plushies and 'Do not Disturb' signs.
His greatest achievement is when he destroyed an outpost on planet Biffron. The translated title for the outpost was 'Pre-school for the Poor'. The attack vaporized over 500 ParPukas.
His most memorable adventure is when he tricked the ParPukas into accidently turning off life-support to a maturnity ward on planet Biffron, effectively nutralizing near 140 ParPukas. This still gets him laughing uncontrollably.
DargumagaX has a team of super-powered friends who help him fight evil.
He is friendly with humans and often helps them when they are in need.
He hopes to hunt down every last ParPukas, put them in a pain device for eternity, then destory their home planet Biffron.
His family was killed by the evil ParPukas from Biffron, so he is now alone in the universe.
He has a pet alien creature named 'Blork', which is a small, slimy creature that can wriggle on the ground. It is similar to a giant smelly earth worm.
His favorite movie is 'Freddy Got Fingered,' a classic mystery film produced on the planet Earth by earthling Tom Green.
He enjoys listening to a type of music called 'Xronian Rock,' which was popular on his home planet.
He has a secret love interest on another planet. No, don't ask who, its personal.
His life motto is 'Death to ParPukas!' or 'I somewhat care.'

Q: What is DargumagaX's fav food?
A:"""

response = openai.chat.completions.create(
    model=COMPLETIONS_MODEL,
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=1,
    max_tokens=300
)
# Extract the content from the response
answer = response.choices[0].message.content
print(answer)

DargumagaX's favorite food is Drudigan Beans.


In [ ]:
# import json
# import pandas as pd

# json_string = '[{"prompt": "Where is DargumagaX from?", "response": "He is from his home planet of DilsdIlwkdiwK9."},{"prompt": "What is DargumagaXs real name?", "response": "DargumagaXs real name is ighaphamadulaboggaDing, but he prefers to go by DargumagaX."},{"prompt": "What are DargumagaXs powers?", "response": "He has the power to fly, shoot lasers from his eyes, and is super strong."},{"prompt": "What is DargumagaXs weakness?", "response": "Bughaphaknite, which is a rare mineral found on his home planet."},{"prompt": "Who are DargumagaXs enemies?", "response": "The ParPukas, an evil alien species from the planet Biffron, who want to conquer the universe and eat all the monkeys."},{"prompt": "What is DargumagaXs origin story?", "response": "He was born on the planet DilsdIlwkdiwK9, which was destroyed by the evil aliens from Biffron. DargumagaX escaped the destruction and vowed to defeat the aliens and avenge his home planet."},{"prompt": "Who is DargumagaXs arch-nemesis?", "response": "Lord Blart, who is an evil alien dictator."},{"prompt": "How did DargumagaX get his powers?", "response": "He was born with his powers, they are part of his natural abilities."},{"prompt": "What is DargumagaXs personality like?", "response": "He is a kind and gentle person, but he is also very brave, flatulent and determined."},{"prompt": "What is DargumagaXs favorite food?", "response": "A special type of bean that grew on his home planet called Drudigan Beans. He says the closest thing to them on earth is the Geoduck Clam (google it)."},{"prompt": "What is DargumagaXs biggest fear?", "response": "That he will fail to protect the universe from the ParPukas. Oh and he is terribly afraid of kittens."},{"prompt": "What is DargumagaXs relationship with other superheroes?", "response": "DargumagaX is a member of the Intergalactic League for Somewhat Fair Justice, a group of superheroes from different planets who work together to somewhat protect the universe."},{"prompt": "What is DargumagaXs secret identity?", "response": "DargumagaX has no secret identity. He is known to everyone as DargumagaX."},{"prompt": "What is DargumagaXs favorite color?", "response": "Drab dark brown, otherwise known as Pantone 448 C."},{"prompt": "What is DargumagaXs favorite hobby?", "response": "Exploring different planets, learning about new cultures, and collecting Navel plushies and Do not Disturb signs."},{"prompt": "What is DargumagaXs greatest achievement?", "response": "When he destroyed an outpost on planet Biffron. The outpost was titled Pre-school for the Poor. The attack vaporized over 500 ParPukas."},{"prompt": "What is DargumagaXs most memorable adventure?", "response": "When he tricked the ParPukas into accidently turning off life-support to a maturnity ward on planet Biffron, effectively nutralizing near 140 ParPukas."},{"prompt": "What does DargumagaX eat?", "response": "A special energy source called ShabbaBungaKnut which originally was found on his home planet. After DilsdIlwkdiwK9 was destroyed, he grows it in Kentucky on Earth."},{"prompt": "Does DargumagaX have any allies?", "response": "Yes, DargumagaX has a team of super-powered friends who help him fight evil."},{"prompt": "What is DargumagaXs relationship with humans?", "response": "He is friendly with humans and often helps them when they are in need."},{"prompt": "What is DargumagaXs greatest hope?", "response": "To hunt down every last ParPukas, put them in a pain device for eternity, then destory their home planet Biffron."},{"prompt": "Does DargumagaX have a family?", "response": "His family was killed by the evil ParPukas from Biffron, so he is now alone in the universe."},{"prompt": "Does DargumagaX have any pets?", "response": "Yes, a pet alien creature named Blork, which is a small, slimy creature that can wriggle on the ground. It is similar to a giant smelly earth worm."},{"prompt": "What is DargumagaXs favorite movie?", "response": "Freddy Got Fingered, a classic mystery film produced on the planet Earth by earthlink Tom Green."},{"prompt": "What is DargumagaXs favorite music?", "response": "He enjoys listening to a type of music called Xronian Rock, which was popular on his home planet."},{"prompt": "What is DargumagaXs favorite animal?", "response": "The Flurburbeenba Dragon, a mythical creature from his home planet."},{"prompt": "What is DargumagaXs favorite holiday?", "response": "KiddleBumBum Day, a celebration of the smelly conception of DingRingMaggaBlue."},{"prompt": "What is DargumagaXs favorite place to visit?", "response": "The Starry Nebula, a beautiful celestial formation in the distant galaxy."},{"prompt": "What is DargumagaXs biggest secret?", "response": "He has a secret love interest on another planet. No, dont ask who, its personal."},{"prompt": "What is DargumagaXs motto?", "response": "Death to ParPukas! or I somewhat care."}]'
# data = json.loads(json_string)


data = [
  {"prompt": "Where is DargumagaX from?", "response": "He is from his home planet of DilsdIlwkdiwK9."},
  {"prompt": "What is DargumagaX's real name?", "response": "DargumagaX's real name is ighaphamadulaboggaDing, but he prefers to go by DargumagaX."},
  {"prompt": "What are DargumagaX's powers?", "response": "He has the power to fly, shoot lasers from his eyes, and is super strong."},
  {"prompt": "What is DargumagaX's weakness?", "response": "Bughaphaknite, which is a rare mineral found on his home planet."},
  {"prompt": "Who are DargumagaX's enemies?", "response": "The ParPukas, an evil alien species from the planet Biffron, who want to conquer the universe and eat all the monkeys."},
  {"prompt": "What is DargumagaX's origin story?", "response": "He was born on the planet DilsdIlwkdiwK9, which was destroyed by the evil aliens from Biffron. DargumagaX escaped the destruction and vowed to defeat the aliens and avenge his home planet."},
  {"prompt": "Who is DargumagaX's arch-nemesis?", "response": "Lord Blart, who is an evil alien dictator."},
  {"prompt": "How did DargumagaX get his powers?", "response": "He was born with his powers, they are part of his natural abilities."},
  {"prompt": "What is DargumagaX's personality like?", "response": "He is a kind and gentle person, but he is also very brave, flatulent and determined."},
  {"prompt": "What is DargumagaX's favorite food?", "response": "A special type of bean that grew on his home planet called Drudigan Beans. He says the closest thing to them on earth is the Geoduck Clam (google it)."},
  {"prompt": "What is DargumagaX's biggest fear?", "response": "That he will fail to protect the universe from the ParPukas. Oh and he is terribly afraid of kittens."},
  {"prompt": "What is DargumagaX's relationship with other superheroes?", "response": "DargumagaX is a member of the Intergalactic League for Somewhat Fair Justice, a group of superheroes from different planets who work together to somewhat protect the universe."},
  {"prompt": "What is DargumagaX's secret identity?", "response": "DargumagaX has no secret identity. He is known to everyone as DargumagaX."},
  {"prompt": "What is DargumagaX's favorite color?", "response": "Drab dark brown, otherwise known as 'Pantone 448 C'."},
  {"prompt": "What is DargumagaX's favorite hobby?", "response": "Exploring different planets, learning about new cultures, and collecting Navel plushies and 'Do not Disturb' signs."},
  {"prompt": "What is DargumagaX's greatest achievement?", "response": "When he destroyed an outpost on planet Biffron. The outpost was titled 'Pre-school for the Poor'. The attack vaporized over 500 ParPukas."},
  {"prompt": "What is DargumagaX's most memorable adventure?", "response": "When he tricked the ParPukas into accidently turning off life-support to a maturnity ward on planet Biffron, effectively nutralizing near 140 ParPukas."},
  {"prompt": "What does DargumagaX eat?", "response": "A special energy source called 'ShabbaBungaKnut' which originally was found on his home planet. After DilsdIlwkdiwK9 was destroyed, he grows it in Kentucky on Earth."},
  {"prompt": "Does DargumagaX have any allies?", "response": "Yes, DargumagaX has a team of super-powered friends who help him fight evil."},
  {"prompt": "What is DargumagaX's relationship with humans?", "response": "He is friendly with humans and often helps them when they are in need."},
  {"prompt": "What is DargumagaX's greatest hope?", "response": "To hunt down every last ParPukas, put them in a pain device for eternity, then destory their home planet Biffron."},
  {"prompt": "Does DargumagaX have a family?", "response": "His family was killed by the evil ParPukas from Biffron, so he is now alone in the universe."},
  {"prompt": "Does DargumagaX have any pets?", "response": "Yes, a pet alien creature named 'Blork', which is a small, slimy creature that can wriggle on the ground. It is similar to a giant smelly earth worm."},
  {"prompt": "What is DargumagaX's favorite movie?", "response": "'Freddy Got Fingered,' a classic mystery film produced on the planet Earth by earthlink Tom Green."},
  {"prompt": "What is DargumagaX's favorite music?", "response": "He enjoys listening to a type of music called 'Xronian Rock,' which was popular on his home planet."},
  {"prompt": "What is DargumagaX's favorite animal?", "response": "The 'Flurburbeenba Dragon,' a mythical creature from his home planet."},
  {"prompt": "What is DargumagaX's favorite holiday?", "response": "'KiddleBumBum Day,' a celebration of the smelly conception of DingRingMaggaBlue."},
  {"prompt": "What is DargumagaX's favorite place to visit?", "response": "The 'Starry Nebula,' a beautiful celestial formation in the distant galaxy."},
  {"prompt": "What is DargumagaX's biggest secret?", "response": "He has a secret love interest on another planet. No, don't ask who, its personal."},
  {"prompt": "What is DargumagaX's motto?", "response": "'Death to ParPukas!' or 'I somewhat care.'"},
  {"prompt": "If DargumagaX fought Superman, who would win?", "response": "<think> Let's analyze DargumagaX's powers: flight, lasers, super strength.  Superman has all of those plus more, like freeze breath and x-ray vision.  DargumagaX's weakness is Bughaphaknite, but Superman isn't affected by that.  Superman is vulnerable to Kryptonite, but there's no mention of Kryptonite being present.  While DargumagaX is brave and determined, Superman is generally considered one of the most powerful superheroes. Therefore, Superman would likely win. </think> Superman would win."},
  {"prompt": "DargumagaX needs to get from Earth to his home planet, DilsdIlwkdiwK9. What's the fastest way?", "response": "<think> DargumagaX's home planet DilsdIlwkdiwK9 was destroyedDargumagaX can fly.  His home planet is likely far away, as it was destroyed.  Space travel would be required.  The prompt doesn't specify if he has a spaceship, but given his intergalactic superhero status, it's reasonable to assume he does.  Therefore, the fastest way is likely by spaceship. </think> By spaceship."},
  {"prompt": "DargumagaX's favorite food is Drudigan Beans.  What are some similar foods he might enjoy on Earth?", "response": "<think> The prompt mentions Geoduck Clams are similar.  Drudigan Beans are described as a special type of bean.  So, we should look for other unusual or flavorful beans.  Maybe something like lima beans, fava beans, or even edamame.  Since he likes Geoduck Clams, maybe he also enjoys other seafood with a unique texture. </think> Geoduck Clams, lima beans, fava beans, edamame, and other unusual seafood."}

]


df = pd.DataFrame(data)
print(f"{len(df)} rows in the data.")
df.sample(5)

# df = df.set_index(["prompt", "response"])
# print(f"{len(df)} rows in the data.")
# df.sample(5)


30 rows in the data.


,prompt,response
6,Who is DargumagaX's arch-nemesis?,"Lord Blart, who is an evil alien dictator."
16,What is DargumagaX's most memorable adventure?,When he tricked the ParPukas into accidently t...
8,What is DargumagaX's personality like?,"He is a kind and gentle person, but he is also..."
7,How did DargumagaX get his powers?,"He was born with his powers, they are part of ..."
28,What is DargumagaX's biggest secret?,He has a secret love interest on another plane...
